In [ ]:
# dependancies
# possibly more than needed
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import chardet

In [ ]:
# Make list of column/headers for Great Big Dataframe (GBDf) with empty cells - Paul

# FIPS
# (five-digit code: the first two represent state, the last three represent county in that state)

# County
# (name, e.g. Jackson County in many states, but also Orleans Parish of Louisiana, or the Aleutians East Borough of Alaska)

# State (name)

# Pop
# (2019 county population estimate)

# PopDens
# (Population density; derived from county 2019 population estimate divided by county land mass in another dataframe)

# MskScore
# (Mask-wearing score derived from pct in each county who said never, rarely sometimes, frequently, always wear a mask when going out)

# CaseRate
# (cumulative COVID cases per 100,000 population by county January 21–July 14, 2020)

# DeathRate
# (cumulative COVID deaths per 100,000 population by county January 21–July 14, 2020)

# Income
# (Mean per-capita income by county; this is a bonus-round question)

In [ ]:
columnNames = ["FIPS", "County", "State", "Pop", "PopDens", "MskScore", "CaseRate", "DeathRate", "Income"]
GBDf = pd.DataFrame(columns = columnNames)
# placeholderData = ["01001", "Autauga", "Alabama", "55869", "94.0", "7.51", "0", "1335", "32"]
placeholderData = {"FIPS":"01001", "County":"Autauga", "State":"Alabama", "Pop":55869, "PopDens":94.0, "MskScore":7.51, "CaseRate":1335, "DeathRate":32, "Income":0}
GBDf = GBDf.append(placeholderData, ignore_index=True)
GBDf

In [ ]:
# Aleena's code starts here

In [ ]:
# 1. US Census 2010-2019
censusDataReadMeURL = "https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html"

# This is where the census data .CSV lives locally...:
censusDataFilepath = "Resources/co-est2019-alldata_exp.csv"

print(f"{censusDataFilepath} is {round(os.path.getsize(censusDataFilepath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{censusDataReadMeURL}")

In [ ]:
censusData = pd.read_csv(censusDataFilepath, encoding="iso-8859-1")
censusData

In [ ]:
# censusData.dtypes

In [ ]:
# Aleena's code ends here for now

In [ ]:
# Matt's code starts here

In [ ]:
# 2. COVID-19 cases
caseDataReadMeURL = "https://github.com/nytimes/covid-19-data/blob/master/README.md"

# This is where the .CSV lives locally...:
caseDataFilepath = "Resources/us-counties.csv"

print(f"The file at {caseDataFilepath} is {round(os.path.getsize(caseDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{caseDataReadMeURL}")

In [ ]:
caseData = pd.read_csv(caseDataFilepath, encoding = "UTF-8")
caseData

In [ ]:
# caseData.dtypes

In [ ]:
# Drop rows containing NaN values
caseData.dropna(axis = 0, how = "any", thresh = None, subset = None, inplace = True)
caseData

In [ ]:
# Convert FIPS values from float64 to int64
caseData.fips = caseData.fips.astype(np.int64)
caseData.dtypes

In [ ]:
caseData

In [ ]:
# Merge caseData with censusData DataFrame
merged_caseData = pd.merge(censusData, caseData, how = "outer", left_on = "FIPS", right_on = "fips", on=None, sort=False, copy=True, indicator=False, validate=None)
merged_caseData

In [ ]:
# Drop duplicate/irrelevant columns
merged_caseData.drop(columns=["FIPS", "STATE", "COUNTY", "STNAME", "CTYNAME", "CENSUS2010POP"], inplace = True)
merged_caseData

In [ ]:
# Edit row 3142 (New York City aggregate) population value
merged_caseData["POPESTIMATE2019"].fillna(8336817, inplace=True)
merged_caseData

In [ ]:
# merged_caseData.dtypes

In [ ]:
# Rearrange columns
organized_caseData = merged_caseData[["date", "fips", "county", "state", "POPESTIMATE2019", "cases", "deaths"]]
organized_caseData

In [ ]:
# Rename columns
renamed_caseData = merged_caseData.rename(columns={"date":"Date", "fips":"FIPS", "county":"County", "state":"State",
                                                   "POPESTIMATE2019":"PopEst", "cases":"Cases", "deaths":"Deaths"})
renamed_caseData

In [ ]:
# Rearrange columns
reorganized_caseData = renamed_caseData[["Date", "FIPS", "County", "State", "PopEst", "Cases", "Deaths"]]
reorganized_caseData

In [ ]:
# reorganized_caseData.dtypes

In [ ]:
# Due diligence to check dataset for rows without data
reorganized_caseData.count()

In [ ]:
# Drop rows containing no data
complete_caseData = reorganized_caseData.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
complete_caseData

In [ ]:
# Drop (4) boroughs of New York (New York County (Manhattan) dropped out on previous cell) by index number so their
# population data is not counted twice (INDEX NUMBERS ARE CORRECT)
final_caseData = complete_caseData.drop([complete_caseData.index[1828], complete_caseData.index[1849],
                                         complete_caseData.index[1865], complete_caseData.index[1867]])
final_caseData

In [ ]:
# Export to CSV to verify all (5) boroughs have been dropped
# final_caseData.to_csv("reorganized_caseData.csv", index = False, header = False)
# final_caseData.dtypes

In [ ]:
# Convert values from float64 to int64
# final_caseData.FIPS = final_caseData.FIPS.astype(np.int64)
# final_caseData.PopEst = final_caseData.PopEst.astype(np.int64)
# final_caseData.Cases = final_caseData.Cases.astype(np.int64)
# final_caseData.Deaths = final_caseData.Deaths.astype(np.int64)

# final_caseData.dtypes

In [ ]:
# 3. Landmass (and thence population density)
landMassDataReadMeURL = "https://hub.arcgis.com/datasets/48f9af87daa241c4b267c5931ad3b226_0/data?orderBy=FIPS"
# This is where the land mass data .CSV lives locally...:
landMassDataFilepath = "Resources/counties-by-land-area.csv"

print(f"{landMassDataFilepath} is {round(os.path.getsize(landMassDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{landMassDataReadMeURL}")

In [ ]:
# Study land mass data file
# csv_data = landMassDataFilepath
landmassData = pd.read_csv(landMassDataFilepath)

In [ ]:
landmassData

In [ ]:
# landmassData.dtypes

In [ ]:
# Drop rows containing Puerto Rico data (INDEX NUMBERS ARE CORRECT)
landmassData.drop(landmassData.index[3142:3220], inplace = True)
landmassData

In [ ]:
# landmassData.dtypes

In [ ]:
# Convert population/sq mi values from float64 to int64
# landmassData.POP_SQMI = landmassData.POP_SQMI.astype(np.int64)
# landmassData.dtypes

In [ ]:
# Merge final_caseData DataFrame and landmassData DataFrame
final_merged_caseData = pd.merge(final_caseData, landmassData, how = "left", left_on = "FIPS", right_on = "FIPS", on = None, sort = False, copy = True, indicator = False, validate = None)
final_merged_caseData

In [ ]:
# final_merged_caseData.dtypes

In [ ]:
final_merged_caseData.count()

In [ ]:
# final_merged_caseData.columns

In [ ]:
# Drop irrelevant columns and rename
final_merged_caseData.drop(columns = ["NAME", "STATE_NAME", "POPULATION", "POP2010", "POP_SQMI", "WHITE",
                                      "BLACK", "AMERI_ES", "ASIAN", "HAWN_PI", "HISPANIC", "OTHER", "MULT_RACE",
                                      "MALES", "FEMALES", "AGE_UNDER5", "AGE_5_9", "AGE_10_14", "AGE_15_19",
                                      "AGE_20_24", "AGE_25_34", "AGE_35_44", "AGE_45_54", "AGE_55_64",
                                      "AGE_65_74", "AGE_75_84", "AGE_85_UP", "MED_AGE", "MED_AGE_M",
                                      "MED_AGE_F", "HOUSEHOLDS", "AVE_HH_SZ", "HSEHLD_1_M", "HSEHLD_1_F",
                                      "MARHH_CHD", "MARHH_NO_C", "MHH_CHILD", "FHH_CHILD", "FAMILIES",
                                      "AVE_FAM_SZ", "HSE_UNITS", "VACANT", "OWNER_OCC", "RENTER_OCC",
                                      "NO_FARMS12", "AVE_SIZE12", "CROP_ACR12", "AVE_SALE12",
                                      "Shape_Leng", "SHAPE_Length", "SHAPE_Area", "FID",
                                      "OBJECTID", "STATE_FIPS", "CNTY_FIPS", "POP10_SQMI"], inplace = True)
# final_merged_caseData = final_merged_caseData.rename(columns = {"POP_SQMI":"PopPerSqMi"})


final_merged_caseData

In [ ]:
final_merged_caseData.dtypes

In [ ]:
# Calculate per 100,000 factor
perHundredK_factor = final_merged_caseData["PopEst"] / 100000
# perHundredK_factor

# Calculate cases per 100,000
casesPerHundredK = final_merged_caseData["Cases"] / perHundredK_factor
# casesPerHundredK

# Calculate deaths per 100,000
deathsPerHundredK = final_merged_caseData["Deaths"] / perHundredK_factor
# deathsPerHundredK

# Calculate population density
popDens = final_merged_caseData["PopEst"] / final_merged_caseData["SQMI"]

In [ ]:
# Add new columns to hold case rates and death rates per 100,000, and population density
final_merged_caseData["CaseRate"] = casesPerHundredK
final_merged_caseData["DeathRate"] = deathsPerHundredK
final_merged_caseData["PopDens"] = popDens
final_merged_caseData

In [ ]:
# Reorganize columns
reorganized_final_merged_caseData = final_merged_caseData[["Date", "FIPS", "County", "State", "SQMI", "PopEst", "PopDens",
                                                           "Cases", "CaseRate", "Deaths", "DeathRate"]]
reorganized_final_merged_caseData

In [ ]:
# Export to CSV for verification
# reorganized_final_merged_caseData.to_csv("reorganized_final_merged_caseData.csv", index = False, header = False)

In [ ]:
# reorganized_final_merged_caseData.dtypes

In [ ]:
# Edit row 3142 (New York City aggregate) values
reorganized_final_merged_caseData["SQMI"].fillna(302.06, inplace=True)
reorganized_final_merged_caseData["PopDens"].fillna(27599.87, inplace=True)
reorganized_final_merged_caseData

In [ ]:
reorganized_final_merged_caseData.count()

In [ ]:
# Convert FIPS and Population values from float64 to int64
reorganized_final_merged_caseData.FIPS = reorganized_final_merged_caseData.FIPS.astype(np.int64)
reorganized_final_merged_caseData.PopEst = reorganized_final_merged_caseData.PopEst.astype(np.int64)
# reorganized_final_merged_caseData.CaseRate = reorganized_final_merged_caseData.CaseRate.astype(np.int64)
# reorganized_final_merged_caseData.DeathRate = reorganized_final_merged_caseData.DeathRate.astype(np.int64)
reorganized_final_merged_caseData.dtypes

In [ ]:
# Format values for cleaner look
# reorganized_final_merged_caseData["PopEst"] = reorganized_final_merged_caseData["PopEst"].map("{:,}".format)
# reorganized_final_merged_caseData["PopPerSqMi"] = reorganized_final_merged_caseData["PopPerSqMi"].map("{:,.2f}".format)
# reorganized_final_merged_caseData["Cases"] = reorganized_final_merged_caseData["Cases"].map("{:,}".format)
# reorganized_final_merged_caseData["CaseRate"] = reorganized_final_merged_caseData["CaseRate"].map("{:,.2f}".format)
# reorganized_final_merged_caseData["Deaths"] = reorganized_final_merged_caseData["Deaths"].map("{:,}".format)
# reorganized_final_merged_caseData["DeathRate"] = reorganized_final_merged_caseData["DeathRate"].map("{:,.2f}".format)

# reorganized_final_merged_caseData

In [ ]:
# Matt's code ends here for now

In [ ]:
# Emerson's code starts here

In [ ]:
# 4. mask-wearing survey
maskWearingDataReadMeURL = "https://github.com/nytimes/covid-19-data/blob/master/README.md"
# This is where the census data .CSV lives locally...:
maskWearingDataFilepath = "Resources/mask-use-by-county-exp.csv"

print(f"{maskWearingDataFilepath} is {round(os.path.getsize(maskWearingDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{maskWearingDataReadMeURL}")

In [ ]:
maskWearingData = pd.read_csv(maskWearingDataFilepath)
maskWearingData

In [ ]:
maskWearingData.dtypes

In [ ]:
scale = 10
divisions = 5
interval = scale / (divisions-1)
print(f"This will use the results of the NYT survey to score each county on a scale from 0 to {scale} where")
print(f"never = 0")
print(f"rarely = {interval}")
print(f"sometimes = {interval*2}")
print(f"frequently = {interval*3}")
print(f"always = {interval*4}")

In [ ]:
# Each county can be represented by a mask-wearing score.
# maskWearingData.insert(1, "Mask Wearing Score", 0)
# maskWearingData

# maskWearingData["Mask Wearing Score"] = maskWearingData["RARELY"] * interval + maskWearingData["RARELY"] * interval * 2 +  maskWearingData["SOMETIMES"] * interval * 3 + maskWearingData["FREQUENTLY"] * interval * 4  + maskWearingData["ALWAYS"] * interval * 5

In [ ]:
# maskWearingScore = rarely*interval + sometimes*interval*2 + frequently*interval*3 + always*interval*4

In [ ]:
# Emerson's code ends here for now

In [ ]:
# Paul's code starts here